In [1]:
import pandas as pd
import numpy as np
from queue import PriorityQueue
from dataclasses import dataclass

In [2]:
grid = (
    pd.read_csv("data/16.txt", engine="python", sep=r"\s*", names=range(-1, 1000))
    .dropna(axis=1, how="all")
    .values
)

In [3]:
@dataclass(frozen=True, order=True)
class NodeState:
    position: tuple
    direction: tuple


def enhanced_heuristic(x, y, goal_x, goal_y):
    return min(np.abs(x - goal_x), np.abs(y - goal_y))


def get_neighbors(node_state: NodeState):
    x, y = node_state.position
    dx, dy = node_state.direction
    if 0 <= x + dx < grid.shape[0] and 0 <= y + dy < grid.shape[1]:
        if grid[x + dx, y + dy] != "#":
            yield NodeState(position=(x + dx, y + dy), direction=(dx, dy)), 1

    yield NodeState(position=(x, y), direction=(dy, -dx)), 1000
    yield NodeState(position=(x, y), direction=(-dy, dx)), 1000

In [4]:
start = np.where(grid == "S")
start_x, start_y = start[0][0], start[1][0]

goal = np.where(grid == "E")
goal_x, goal_y = goal[0][0], goal[1][0]

frontier = PriorityQueue()
frontier.put((0, NodeState(position=(start_x, start_y), direction=(0, 1))))

cost_so_far = {start_state: 0 for _, start_state in frontier.queue}
came_from = {start_state: None for _, start_state in frontier.queue}

while not frontier.empty():
    _, current = frontier.get()
    if current.position == goal:
        break

    for next_state, price in get_neighbors(current):
        new_cost = cost_so_far[current] + price
        if next_state not in cost_so_far or new_cost < cost_so_far[next_state]:
            cost_so_far[next_state] = new_cost
            priority = new_cost + enhanced_heuristic(
                *next_state.position, goal_x, goal_y
            )
            frontier.put((priority, next_state))
            came_from[next_state] = current

cost_so_far[current]

134588